<a href="https://colab.research.google.com/github/Atrib99/Python_Projects/blob/master/Inventory_Optimization_for_quick_commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install dash


In [6]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output

# Load pre-generated data
orders_df = pd.read_csv('orders_data.csv')
avg_demand_df = pd.read_csv('inventory_plan.csv')

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Quick Commerce Inventory Dashboard"

# Layout
app.layout = html.Div([
    html.H1("Inventory Optimization Dashboard", style={"textAlign": "center"}),

    dcc.Dropdown(
        id='pincode-dropdown',
        options=[{'label': pin, 'value': pin} for pin in avg_demand_df['Pincode'].unique()],
        value='11000',
        clearable=False,
        style={"width": "300px", "margin": "auto"}
    ),

    html.Br(),

    dcc.Graph(id='demand-trend'),
    dcc.Graph(id='inventory-plan')
])

# Callbacks
@app.callback(
    [Output('demand-trend', 'figure'),
     Output('inventory-plan', 'figure')],
    [Input('pincode-dropdown', 'value')]
)
def update_dashboard(selected_pincode):
    # Historical Demand Trend
    trend_df = orders_df[orders_df['Pincode'] == selected_pincode]
    trend_df = trend_df.groupby(['Date', 'Product'])['Orders'].sum().reset_index()
    trend_fig = px.line(trend_df, x='Date', y='Orders', color='Product',
                        title=f"Daily Orders Trend - Pincode {selected_pincode}")

    # Inventory Plan
    inventory_df = avg_demand_df[avg_demand_df['Pincode'] == selected_pincode]
    inventory_fig = px.bar(inventory_df, x='Product', y='Optimal_Weekly_Stock',
                           color='Product', text='Optimal_Weekly_Stock',
                           title=f"Optimal Weekly Stock Plan - Pincode {selected_pincode}")
    inventory_fig.update_traces(textposition='outside')

    return trend_fig, inventory_fig

# Run the server
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>